In [ ]:
import pickle
import json
from pathlib import Path
from slideminer import data

In [ ]:
DATA = Path('/data/DeepLearning/SCHWOB_Robin/Seg_outputs_2_iter_10_all_images')
ITER = 3
TREE = DATA/'Tree/most_significant_tree_final.p'
PATCHES = DATA/f'Selection'

In [ ]:
with open(TREE, 'rb') as f:
    p_tree = pickle.load(f)

In [ ]:
root = max(p_tree.children.keys())

In [ ]:
p_tree.population[root]

167671

In [ ]:
max(p_tree.children.keys()), p_tree.children[max(p_tree.children.keys())], [p_tree.population[c] for c in p_tree.children[root]]

(335340, [214689, 214729], [18642, 149029])

In [ ]:
with open(TREE.parent/'full_tree.p', 'rb') as f:
    f_tree = pickle.load(f)

In [ ]:
root, f_tree.children[root], [f_tree.population[c] for c in f_tree.children[root]]

(249462, (113313, 249461), [1, 124731])

In [ ]:
truncs = [int(node.name) for node in PATCHES.iterdir()]; len(truncs)

62

In [ ]:
leaves_by_trunc = data.leaves_under_truncated_leaves(p_tree, truncs)

In [ ]:
all_stats = {}
for concept in PATCHES.iterdir():
    all_stats[int(concept.name)] = {'04': 0, '05': 0, '08': 0}
    for patch in concept.iterdir():
        for pacs in ('04', '05', '08'):
            if f'PACS{pacs}' in patch.name:
                all_stats[int(concept.name)][pacs] += 1
                break

In [ ]:
def get_pacs_stats(concept):
    if concept in all_stats:
        return all_stats[concept]
    else:
        count = {'04': 0, '05': 0, '08': 0}
        for child in p_tree.children[concept]:
            child_count = get_pacs_stats(child)
            for pacs in count:
                count[pacs] += child_count[pacs]
        return count

In [ ]:
def compute_dict(nodes):
    parents = []
    for node in nodes:
        node = int(node)
        try:
            parent = int(p_tree.parents[node])
        except:
            parent = node
        test = node != parent
        if test:
            jsondict['links'].append({'source': parent, 'target': node})
        dico = {'id': node,
                'parent': parent}
        if node in all_stats:
            dico['children'] = []
            count = all_stats[node]
            tot = sum(count.values())
            dico['leaf_size_color'] = [count[p] / tot for p in ('04', '05', '08')] + [1.0]
        else:
            dico['children'] = [int(c) for c in p_tree.children[node] if str(c) in jsondict['nodes']]
            n = len(dico['children'])
            dico['leaf_size_color'] = [0, 0, 0, 0]
            for child in dico['children']:
                child_color = jsondict['nodes'][str(child)]['leaf_size_color']
                for k, c in enumerate(child_color):
                    dico['leaf_size_color'][k] += c/n
        jsondict['nodes'][str(node)] = dico
        if test and parent not in parents:
            parents.append(parent)
    if len(parents):
        compute_dict(parents)

In [ ]:
jsondict = {'links': [], 'nodes': {}}
compute_dict(leaves_by_trunc)

In [ ]:
with open(TREE.with_suffix('.json'), "w") as f:
    json.dump(jsondict, f, indent=4, sort_keys=True)